In [1]:
from PandaStation import (
    PandaStation, FindResource, AddPackagePaths, AddRgbdSensors, draw_points, draw_open3d_point_cloud, 
    create_open3d_point_cloud)

from PandaGrasping import *

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

# Let's do all of our imports here, too.
import numpy as np
import ipywidgets
import pydot
import pydrake.all
import os
from IPython.display import display, SVG
import open3d as o3d
import meshcat
import meshcat.geometry as g
import meshcat.transformations as tf


import pydrake.all
from pydrake.geometry import Cylinder, Box
from pydrake.all import (
    RigidTransform, RotationMatrix, AngleAxis, RollPitchYaw, InverseKinematics, MultibodyPlant, Parser,
    FindResourceOrThrow, Solve, PiecewisePolynomial, TrajectorySource, SceneGraph, DiagramBuilder,
    AddMultibodyPlantSceneGraph, LinearBushingRollPitchYaw, MathematicalProgram, AutoDiffXd, GenerateHtml, Role,
    MakeRenderEngineVtk, DepthRenderCamera, RenderCameraCore, CameraInfo, ClippingRange,  DepthImageToPointCloud,
    BaseField, RenderEngineVtkParams, ConnectMeshcatVisualizer, DepthRange, RgbdSensor, MeshcatPointCloudVisualizer,
    LoadModelDirectives, ProcessModelDirectives, Box, Sphere, Cylinder
    )
from PandaInverseKinematics import PandaInverseKinematics, PandaIKTraj, Waypoint, Trajectory
from RRT import PandaRRTPlanner, PandaRRTompl
from collections import OrderedDict

import matplotlib.pyplot as plt

ycb = {"cracker": "drake/manipulation/models/ycb/sdf/003_cracker_box.sdf", 
    "sugar": "drake/manipulation/models/ycb/sdf/004_sugar_box.sdf", 
    "soup": "drake/manipulation/models/ycb/sdf/005_tomato_soup_can.sdf", 
    "mustard": "drake/manipulation/models/ycb/sdf/006_mustard_bottle.sdf", 
    "gelatin": "drake/manipulation/models/ycb/sdf/009_gelatin_box.sdf", 
    "meat": "drake/manipulation/models/ycb/sdf/010_potted_meat_can.sdf",
    "brick": "drake/examples/manipulation_station/models/061_foam_brick.sdf"}

for key in ycb.keys():
    ycb[key] = FindResourceOrThrow(ycb[key])
    
ycb["puck"] = FindResource("models/puck.urdf")
ycb["marble"] = FindResource("models/marble.urdf")
ycb["dumbbell"] = FindResource("models/dumbbell.urdf")

def ycb_resource(name):
    global ycb
    return ycb[name]

def random_ycb_resource():
    global ycb
    ycb_items = list(ycb.items())
    index = np.random.randint(0, len(ycb_items))
    return ycb_resource(ycb_items[index][0])

def random_position():
    return [0.6 + np.random.uniform(-0.05, 0.05), np.random.uniform(-0.1,0.1), 0.2]

def random_orientation():
    rot = RotationMatrix(RollPitchYaw(np.random.uniform(-np.pi, np.pi),
                                     np.random.uniform(-np.pi, np.pi),
                                     np.random.uniform(-np.pi, np.pi)))
    return rot

In [2]:
builder = DiagramBuilder()

station = builder.AddSystem(PandaStation())
station.SetupTableStation(welded_hand = True)

station.AddModelFromFile(ycb_resource("brick"),# RigidTransform(random_orientation(), random_position()))
                         RigidTransform(random_orientation(), random_position()))

station.Finalize()

station_context = station.CreateDefaultContext()
scene_graph = station.get_scene_graph()
scene_graph_context = station.GetSubsystemContext(scene_graph, station_context)
plant = station.get_multibody_plant()
plant_context = station.GetSubsystemContext(plant, station_context)
panda = station.GetPanda()
hand = station.GetHand()

v = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder,
          scene_graph,
          output_port=station.GetOutputPort("query_object"),
          delete_prefix_on_load=True,                                      
          zmq_url=zmq_url, role = Role.kProximity)

t1 = time.time()
ik_station, body_infos = create_welded_station(station, station_context)
ik_station_context = ik_station.CreateDefaultContext()

#for shape in body_infos[0].shape_infos:
#    print(shape)
q, cost = grasp_pose(body_infos[0], ik_station, ik_station_context)
print(f"q: {q}\ncost {cost}")
print(f"time: {time.time() - t1}")

diagram = builder.Build()
context = diagram.CreateDefaultContext()

#if np.isfinite(cost):
plant_context = plant.GetMyContextFromRoot(context)
plant.SetPositions(plant_context, panda, q)

v.load()
diagram.Publish(context)

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6027...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7027/static/
Connected to meshcat-server.
q: [ 0.31763777  0.51723645 -0.1080774  -1.57911476  2.8973      3.7525
  0.13978172]
cost -0.008806744412812454
time: 2.385355234146118


In [3]:
#file = open("file.html", "w")
#file.write(v.vis.static_html())
#file.close()

#print(subprocess.check_output(["tar", "vczf", "media/pictures/sphere.gz", "file.html"]))
#print(subprocess.check_output(["trash", "file.html"]))